In [ ]:
# What we want to do:
#1) Load two models together--> Loading happens through a session which restores variables, etc. 
#2) We wish to just load the encoder variables (pretrained) and add them to our computation for composite model
 

#What we try doing is based on the following logic:
#1) A graph defines all operations and data flow. 
#2) However, given that a graph can be invoked through a session, we will need to first create a session. 
#3) QUes: Can a graph only be invoked through a session

#4) Then we load this graph and import the encoder part of it into our composite graph.
#5) We finally run a session for this composite graph. 



#6) HOWEVER, we need to make sure that the encoder is not trained any further. 
#7) Otherwise, we will continue with just two separate sessions. 


'''
The final procedure:
1) Create session to load pretrained graph. Store the graph. Quite session. We do this only once. A graph has to be declared in advance. 
2) Set the encoder weights to be non trainable. 
3) Create new session to train original model and use encoder as a part of the operation flow. 


'''




#For optimization and graph construction:
https://www.kdnuggets.com/2017/05/how-not-program-tensorflow-graph.html
    
#General overview of graphs
https://www.tensorflow.org/programmers_guide/graphs#visualizing_your_graph
    
#Making non trainable variables
https://stackoverflow.com/questions/37326002/is-it-possible-to-make-a-trainable-variable-not-trainable?rq=1
https://stackoverflow.com/questions/35298326/freeze-some-variables-scopes-in-tensorflow-stop-gradient-vs-passing-variables
    
#Loading multiple graphs:
https://stackoverflow.com/questions/41990014/load-multiple-models-in-tensorflow
    
#Loading trained weights from one graph to another
https://stackoverflow.com/questions/39068703/tensorflow-using-weights-trained-in-one-model-inside-another-different-model?rq=1

In [1]:
import os
os.chdir('../model/')
import tensorflow as tf
import pickle
import numpy as np
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
encoder_cell = LSTMCell(encoder_hidden_units)
((encoder_fw_outputs,
          encoder_bw_outputs),
         (encoder_fw_final_state,
          encoder_bw_final_state)) = (
            tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell,
                                    cell_bw=encoder_cell,
                                    inputs=encoder_inputs_embedded,
                                    sequence_length=encoder_inputs_length,
                                    dtype=tf.float32, time_major=True)
            ) #'
    
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs),2)
    

encoder_final_state_c = tf.concat(
        (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
        (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

encoder_final_state = LSTMStateTuple(
        c=encoder_final_state_c,
        h=encoder_final_state_h
    ) #this is useful later

encoder_concat_everything = tf.concat([encoder_final_state_c,encoder_final_state_h], 1)